In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.metrics as mx
from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split,KFold, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import roc_curve, plot_roc_curve, confusion_matrix, roc_auc_score, f1_score, accuracy_score, balanced_accuracy_score, classification_report,RocCurveDisplay,auc
from sklearn.svm import SVC, LinearSVC, NuSVC, SVR, NuSVR, LinearSVR
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier 
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import RepeatedKFold
from sklearn import model_selection
from sklearn.preprocessing import scale 

from numpy import mean, std

from sklearn.cluster import DBSCAN, OPTICS
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from sklearn.inspection import permutation_importance


In [2]:
circRNA_HCC=pd.read_table('HCC_circRNAs.txt').T
circRNA_HCC.columns=circRNA_HCC.iloc[0]
circRNA_HCC=circRNA_HCC.iloc[1:,:]

circRNA_Healthy=pd.read_table('Healthy_circRNAs.txt').T
circRNA_Healthy.columns=circRNA_Healthy.iloc[0]
circRNA_Healthy=circRNA_Healthy.iloc[1:,:]


circRNA=pd.concat([circRNA_HCC,circRNA_Healthy],axis=0)

longRNA_HCC=pd.read_table('HCC_longRNAs.txt').T
longRNA_HCC.columns=longRNA_HCC.iloc[0]
longRNA_HCC=longRNA_HCC.iloc[1:,:]

longRNA_Healthy=pd.read_table('Healthy_longRNAs.txt').T
longRNA_Healthy.columns=longRNA_Healthy.iloc[0]
longRNA_Healthy=longRNA_Healthy.iloc[1:,:]

longRNA_HCC['Group']='1'
longRNA_Healthy['Group']='0'
longRNA=pd.concat([longRNA_HCC,longRNA_Healthy],axis=0)

full_data=pd.concat([circRNA,longRNA], axis=1, ignore_index=True)
full_data.columns=circRNA.columns.append(longRNA.columns)


FileNotFoundError: [Errno 2] No such file or directory: 'HCC_circRNAs.txt'

In [ ]:
Y=full_data.iloc[:,full_data.columns == 'Group']
X=full_data.iloc[:,full_data.columns != 'Group']

In [ ]:
DGE=pd.read_csv('DGE.csv')

In [ ]:
DGE

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size= 0.20, random_state=1303,stratify=Y,shuffle = True)

In [ ]:
Xtr_filtered=X_tr[X_tr.columns.intersection(DGE['Gene'])]



Xtr_filtered=Xtr_filtered.fillna(0)

In [ ]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
Xtr_filtered_S= transformer.fit_transform(Xtr_filtered)

In [ ]:
def modelPerformance(confMat):
    TN = confMat[0, 0]
    TP = confMat[1, 1]
    FP = confMat[0, 1]
    FN = confMat[1, 0]
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    spec = TN / (TN + FP)
    fpr = FP / (TN + FP)
    f1 = 2 * (prec * rec) / (prec + rec)
    acc = (TP + TN) / (TP + FP + TN + FN)
    return (acc, prec, rec, spec, fpr, f1)

def printPerformance(confMat):
    acc, prec, rec, spec, fpr, f1 = modelPerformance(confMat)
    print("Accuracy = " "%.4f" % acc)
    print("Precision = " "%.4f" % prec)
    print("Recall = " "%.4f" % rec)
    print("Specificity = " "%.4f" % spec)
    print("False positive rate = " "%.4f" % fpr)
    print("F1-score = " "%.4f" % f1)
    np.set_printoptions(precision=2)
    print("Confusion matrix (%):")
    print(confMat/np.sum(confMat)*100)

In [ ]:
parameters = {'solver': ['lbfgs','adam','sgd'], 'max_iter': [1000], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':[(100,200)] ,'random_state':[1303], 'activation':['tanh','relu','logistic']}


gridcv = GridSearchCV(MLPClassifier(), parameters, refit=True, cv=5, verbose=1, n_jobs=-1)


# fit the model for grid search 
gridcv.fit(Xtr_filtered_S, Y_tr) 
 
# display best parameters after tuning 
display(gridcv.best_params_) 


In [ ]:
mlp = gridcv.best_estimator_
cv = StratifiedKFold(n_splits=5)

tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    mlp.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel()).predict_proba(Xtr_filtered_S[test])
    vis=plot_roc_curve(mlp, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="MLP")
ax.legend(loc="lower right")
plt.show()


In [ ]:
param_grid_SVC = {'C': np.logspace(-2, 10, 13), 
              'random_state':[1303],
              'kernel':['linear','poly','rbf','sigmoid'],
              'gamma':['scale','auto']}

gridcv = GridSearchCV(SVC(), param_grid_SVC, refit=True, cv=5, verbose=1, n_jobs=-1)


# fit the model for grid search 
gridcv.fit(Xtr_filtered_S, Y_tr) 
 
# display best parameters after tuning 
display(gridcv.best_params_) 


In [ ]:
clfSVC = gridcv.best_estimator_

cv = StratifiedKFold(n_splits=5)
scores = ('accuracy','roc_auc')
output = pd.DataFrame(cross_validate(clfSVC, Xtr_filtered_S, Y_tr, scoring=scores, cv=cv, n_jobs=-1))
print(mean(output['test_roc_auc']))
mean((output['test_accuracy']))

In [ ]:
tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    clfSVC.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel())
    vis=plot_roc_curve(clfSVC, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="SVM poly")
ax.legend(loc="lower right")
plt.show()


In [ ]:
n_estimators = [ 200, 1000, 7000]
max_depth = [5, 8]
min_samples_split = [2, 5, 15]
min_samples_leaf = [1, 2, 5] 
class_weight=['balanced_subsample']
hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf,class_weight=class_weight)

gridF = GridSearchCV(RandomForestClassifier(random_state = 1303), hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
gridF.fit(Xtr_filtered_S, Y_tr)
display(gridF.best_params_) 

In [ ]:
bestF = gridF.best_estimator_
scores = ('accuracy','roc_auc')
output = pd.DataFrame(cross_validate(bestF, Xtr_filtered_S, Y_tr, scoring=scores, cv=cv, n_jobs=-1))
print(mean(output['test_roc_auc']))
mean((output['test_accuracy']))

In [ ]:
tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    bestF.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel())
    vis=plot_roc_curve(bestF, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="Random Forest")
ax.legend(loc="lower right")
plt.show()



In [ ]:
from sklearn.linear_model import LogisticRegression

parameters = {'C': np.logspace(-2, 10, 13), 
              'random_state':[1303],
              'tol':np.logspace(-2, 10, 13)}


gridL = GridSearchCV(LogisticRegression(), parameters, cv = 5, verbose = 1, n_jobs = -1)
bestL = gridL.fit(Xtr_filtered_S, Y_tr)
display(bestL.best_params_) 
bestL.best_score_


In [ ]:

LogReg=bestL.best_estimator_
output=pd.DataFrame(cross_validate(LogReg,Xtr_filtered_S, Y_tr, scoring=['roc_auc','accuracy'],cv=cv,n_jobs=-1))
print(mean(output))


In [ ]:
tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    LogReg.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel())
    vis=plot_roc_curve(LogReg, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="Logistic Regression")
ax.legend(loc="lower right")
plt.show()



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

parameters = {'n_neighbors': range(1,100),
              'weights':['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean','manhattan','l1','l2']}


gridN = GridSearchCV(KNeighborsClassifier(), parameters, cv = 5, verbose = 1, n_jobs = -1)
bestN = gridN.fit(Xtr_filtered_S, Y_tr)
display(bestN.best_params_) 
bestN.best_score_

In [ ]:
KNN=bestN.best_estimator_

tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    KNN.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel())
    vis=plot_roc_curve(KNN, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="K-nearest neighbour")
ax.legend(loc="lower right")
plt.show()



In [ ]:
from sklearn.naive_bayes import GaussianNB

parameters = {'var_smoothing': np.logspace(-2, 10, 13),}


gridG = GridSearchCV(GaussianNB(), parameters, cv = 5, verbose = 1, n_jobs = -1)
bestG = gridG.fit(Xtr_filtered_S, Y_tr)
display(bestN.best_params_) 
bestG.best_score_

In [ ]:
from sklearn.naive_bayes import GaussianNB

GNB=bestG.best_estimator_

tprs=[]
aucs=[]
mean_fpr=np.linspace(0,1,100)
cv = StratifiedKFold(n_splits=5)
fig,ax=plt.subplots()
for i, (train, test) in enumerate(cv.split(Xtr_filtered_S, Y_tr)):
    
    GNB.fit(Xtr_filtered_S[train],Y_tr.iloc[list(train)].values.ravel())
    vis=plot_roc_curve(GNB, Xtr_filtered_S[test],Y_tr.iloc[list(test)], name='ROC fold{}'.format(i),alpha=0.3,lw=1,ax=ax)
   
    tprs.append(np.interp(mean_fpr, vis.fpr, vis.tpr))
    tprs[-1][0]=0.0
    aucs.append(vis.roc_auc)
   
ax.plot([0,1],[0,1],linestyle='--', lw=2, color='r', label = 'Chance', alpha = 0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1]=1
mean_auc=auc(mean_fpr,mean_tpr)
std_auc=np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b', label=r'Mean ROC (AUC=%0.2f $\pm$ %0.2f)' %(mean_auc,std_auc), lw=2, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr+std_tpr,1)
tprs_lower=np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper,color='grey',alpha= 0.2, label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.01,1.01],ylim=[-0.01,1.01],title="Gaussian Naive Bayes")
ax.legend(loc="lower right")
plt.show()



In [ ]:
#final unseen test
X_TEST=X_val[X_tr.columns.intersection(DGE['Gene'])]
#[['exo_circ_79050','exo_circ_79066','exo_circ_71478','exo_circ_11335', 'exo_circ_68977', 'exo_circ_38752','exo_circ_19955','exo_circ_71780','FGB']]
XTEST_S = transformer.transform(X_TEST)


SVM_pred = clfSVC.predict(XTEST_S)
RocCurveDisplay.from_estimator(clfSVC,XTEST_S,Y_val)
ax.set(title="SVM (poly)")

plt.show()
cmat = confusion_matrix(Y_val, SVM_pred)
printPerformance(cmat)

MLP_pred = mlp.predict(XTEST_S)
RocCurveDisplay.from_estimator(mlp,XTEST_S,Y_val)
ax.set(title="MLP")
plt.show()
cmat = confusion_matrix(Y_val, MLP_pred)
printPerformance(cmat)

RF_pred = bestF.predict(XTEST_S)
RocCurveDisplay.from_estimator(bestF,XTEST_S,Y_val)
ax.set(title="Random Forest")
plt.show()
cmat = confusion_matrix(Y_val, RF_pred)
printPerformance(cmat)

LogReg_pred = LogReg.predict(XTEST_S)
RocCurveDisplay.from_estimator(LogReg,XTEST_S,Y_val)
ax.set(title="Logistic Regression")
plt.show()
cmat = confusion_matrix(Y_val, LogReg_pred)
printPerformance(cmat)

KNN_pred = KNN.predict(XTEST_S)
RocCurveDisplay.from_estimator(KNN,XTEST_S,Y_val)
ax.set(title="K Nearest Neighbour")
plt.show()
cmat = confusion_matrix(Y_val, KNN_pred)
printPerformance(cmat)

GNB_pred = GNB.predict(XTEST_S)
RocCurveDisplay.from_estimator(GNB,XTEST_S,Y_val)
ax.set(title="Gaussian Naive Bayes")
plt.show()
cmat = confusion_matrix(Y_val, GNB_pred)
printPerformance(cmat)

In [ ]:
from sklearn.model_selection import permutation_test_score
score_SVM, perm_scores_SVM, pvalue_SVM = permutation_test_score(
    clfSVC,Xtr_filtered_S, np.ravel(Y_tr), scoring="accuracy", cv=cv, n_permutations=1000
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.hist(perm_scores_SVM, bins=20, density=True)
ax.axvline(score_SVM, ls="--", color="r",label=f"Score on original\ndata: {score_SVM:.3f}")
ax.axvline(mean(perm_scores_SVM), ls="--", color="b", label=f"Mean score on \npermutated data: {mean(perm_scores_SVM):.3f}")

#ax.text(0.7, 10, score_label, fontsize=12)
ax.set_xlabel("Accuracy score")
_ = ax.set_ylabel("Probability")
ax.legend(loc="upper right")


In [ ]:
max(perm_scores_SVM)

In [ ]:
from sklearn.model_selection import permutation_test_score
score_R, perm_scores_R, pvalue_R = permutation_test_score(
    bestF,XTEST_S,Y_val, scoring="accuracy", cv=cv, n_permutations=10
)
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.hist(perm_scores_R, density=True)
ax.axvline(score_R, ls="--", color="r",)
score_label = f"Score on original\ndata: {score_R:.2f}\n(p-value: {pvalue_R:.3f})"
ax.text(0.7, 10, score_label, fontsize=12)
ax.set_xlabel("Accuracy score")
_ = ax.set_ylabel("Probability")
